# Initialization

Want to use non-numerical columns here to include in my ML model.
I had left them out previously.
Using pandas get_dummies and cut methods.

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.impute import SimpleImputer, MissingIndicator

In [2]:
train=pd.read_csv('../data/data.csv.zip', compression='zip', index_col=1)
train=train.drop(columns=['Unnamed: 0'])

In [3]:
def model_score(name, model, X, y, val_y, y_pred, icv):
    cvscore = cross_val_score(model, X, y, cv=icv)
    print('Model {0} score: {1:.4f} and std dev: {2:.4f}'.format(name, cvscore.mean(), cvscore.std()))
    print('Variance score: %.4f' % r2_score(val_y, y_pred))
    return cvscore, r2_score(val_y, y_pred)

In [4]:
def model_validation(results):
    fig, ax = plt.subplots()
    ax.boxplot(results[:,1])
    ax.set_xticklabels(results[:,0])
    ax.set_title('Model CV score')
    plt.show()
    
    plt.figure(figsize=(8, 5))
    plt.bar(results[:,0], results[:,2])
    plt.xlabel('Models')
    plt.ylabel('Var score')
    plt.title('Models variance score')
    plt.show()
    
    return 0

In [5]:
def imputation_cv(models, X, y):
    results = []
    train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
    for name, model in models:
        my_pipeline = make_pipeline(SimpleImputer(), model)
        print(my_pipeline)
        my_pipeline.fit(train_X, train_y)
        y_pred = my_pipeline.predict(val_X)
        score, r2 = model_score(name, my_pipeline, X, y, val_y, y_pred, 10)
        results.append((name, score, r2))
    print("Done")
    return results


# Data cleaning

In [6]:
train.columns = train.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
#train=train.drop(columns=['photo', 'flag', 'club_logo'])
#train=train.drop(columns=['release_clause'])


In [7]:
# = pd.get_dummies(train, columns=['position'])


In [8]:
train['value_unit'] = train['value'].str[-1]
train['nvalue'] = np.where(train['value_unit'] == '0', 0,
                          train['value'].str[1:-1].replace(r'[a-zA-Z]',''))
train['nvalue'] = train['nvalue'].astype(float)
train['nvalue'] = np.where(train['value_unit'] == 'M', train['nvalue'], train['nvalue'] / 1000)

train['wage_unit'] = train['wage'].str[-1]
train['nwage'] = np.where(train['wage_unit'] == '0', 0,
                          train['wage'].str[1:-1].replace(r'[a-zA-Z]',''))
train['nwage'] = train['nwage'].astype(float)
train['nwage'] = np.where(train['wage_unit'] == 'K', train['nwage'], train['nwage'] * 1000)


# Feature selection

In [9]:
train.position.nunique()

27

In [10]:
# "cardinality" means the number of unique values in a column.
# We use it as our only way to select categorical columns here. This is convenient, though
# a little arbitrary.
train = pd.get_dummies(train, columns=['position'])
low_cardinality_cols = [cname for cname in train.columns if 
                                train[cname].nunique() <= 10 and
                                train[cname].dtype == "object"]
numeric_cols = [cname for cname in train.columns if 
                                train[cname].dtype in ['int64', 'float64', 'uint8']]
features = low_cardinality_cols + numeric_cols

features.remove('overall')
target = ['overall']

# Define models

In [11]:
models = []
models.append(('LR', linear_model.LinearRegression()))
models.append(('BR', linear_model.BayesianRidge(n_iter=1000)))
models.append(('Huber', linear_model.HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, 
                                                    max_iter=100, tol=1e-05, warm_start=False)))
models.append(('LarsCV', linear_model.LarsCV(max_iter=1000, eps=1.35, cv=10)))
models.append(('LassoCV', linear_model.LassoCV(max_iter=1000, cv=5)))
models.append(('RF', ensemble.RandomForestRegressor(n_estimators=10)))

# Model fitting and prediction

In [12]:
X = train[features].copy()
y = train[target].copy()
X.sample(2)
#features

,preferred_foot,work_rate,body_type,real_face,value_unit,wage_unit,age,potential,special,international_reputation,...,position_RB,position_RCB,position_RCM,position_RDM,position_RF,position_RM,position_RS,position_RW,position_RWB,position_ST
ID,,,,,,,,,,,,,,,,,,,,,
242003,Left,Medium/ Medium,Normal,No,K,K,19,73,1588,1.0,...,0,0,0,0,0,0,0,0,0,0
186595,Right,High/ High,Lean,Yes,M,K,29,70,1892,1.0,...,0,0,0,0,0,1,0,0,0,0


In [13]:
#X.columns[X.isnull().any()].tolist()
#X.columns[X.dtype in ['int64', 'float64', 'uint8']]
#          df.select_dtypes(include='np.numeric')
#X.select_dtypes(include='number').sample(5)

In [14]:
#X.columns[X.isnull().any()]
#cols_with_missing = (col for col in X.columns if X[col].isnull().any())
X_numeric = X.select_dtypes(include='number')
cols_with_missing = X_numeric.columns[X_numeric.isnull().any()].tolist() #& X.select_dtypes(include='number')
# make new columns indicating what will be imputed
#cols_with_missing

In [15]:
for c in cols_with_missing:
    X[c + '_missing'] = X[c].isnull()
    #print(X[c].isnull().sample(5))


In [16]:
results = imputation_cv(models, X, y)


Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)), ('linearregression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])


AttributeError: 'DataFrame' object has no attribute 'dtype'